# Optimal Transport

Note: Github's default LaTeX rendering messes with some of the equations here. Consider viewing the notebook on [NBViewer](https://nbviewer.jupyter.org/github/kheyer/OTDD/blob/main/Algorithms/Optimal%20Transport.ipynb)

## Overview

Optimal transport is a mathematical approach towards measuring the distance between two arbitrary distributions. We can think of the problem this way. Consider two sets of points:

<img src="https://raw.githubusercontent.com/kheyer/OTDD/main/media/distribution.png" height="40%" width="40%">

We want to map each point in the source distribution to a corresponding point in the target distribution, such that the total distance between all coupled points is minimized. Note that in this framework, we cannot simply match each point in the source distribution to the closest point in the target distribution. We need to ensure that all target points are included in the mapping. For this particular distribution, the coupling would look like this:

<img src="https://raw.githubusercontent.com/kheyer/OTDD/main/media/distribution_coupled.png" height="40%" width="40%">

How do we generate this coupling? First we calculate a cost matrix $C$. For this example, our cost is the squared euclidean distance between points. We then run an optimization routine to find the coupling matrix $\pi_{OT}$ such that

\begin{align}
\pi_{\text{OT}} ~=~ \min_{\pi} \sum \text{C}(x,y) \pi(x,y)
\end{align}

<img src="https://raw.githubusercontent.com/kheyer/OTDD/main/media/cost_coupling.png" height="80%" width="80%">

We can extend this idea to continuous distributions by converting the sum to an integral. We solve the problem

\begin{align}
\pi_{\text{OT}} ~=~ \min_{\pi} \int \text{C}(x,y) d\pi(x,y)
\end{align}

<img src="https://raw.githubusercontent.com/kheyer/OTDD/main/media/gaussian.png" height="80%" width="80%">

## Formulas

Now we'll give a more formal definition of this concept.

We define our two samples as discrete measures in $\mathbb{R}^{n}$ with locations $x_{1}, ..., x_{n} \in \chi_{x}$, $y_{1}, ..., y_{m} \in \chi_{y}$, along with weights $\alpha \in P(\chi_{x})$, $\beta \in P(\chi_{y})$.

The weights $\alpha$, $\beta$ are positive values defining a probability distribution over the elements in $\chi_{x}$, $\chi_{y}$

\begin{align}
\alpha ~=~ \sum_{i=1}^{N}\alpha_{i}\delta_{x_{i}},\quad \beta ~=~ \sum_{j=1}^{M}\beta_{j}\delta_{y_{j}}
\end{align}

\begin{align}
\sum_{i=1}^{N}\alpha_{i} ~=~ 1, \quad \sum_{j=1}^{M}\beta_{i} ~=~ 1
\end{align}

Next we define a cost function $c(x_{i}, y_{j})$ which defines the ground cost between $x_{i}$ and $y_{j}$. This can be any valid distance function. We can then construct our cost matrix $C_{i,j}$ which contains the pairwise costs between all points in $\chi_{x}$ and $\chi_{y}$.

We can now define the optimal transport problem with the Kantorovich formulation:

\begin{align}
\text{OT}(\alpha, \beta) ~=~ \min_{x\in\Pi(\alpha, \beta)}\int_{\chi_{x} \times \chi_{y}} c(x,y)d\pi(x,y)
\end{align}

\begin{align}
\forall i,j, \pi_{i,j} \geq 0, \sum_{j} \pi_{i,j} ~=~ \alpha_{i}, \sum_{i} \pi_{i,j} ~=~ \beta_{j}
\end{align}


We can also define this in probabalistic terms:

\begin{align}
\text{OT}(\alpha, \beta) ~=~ \min_{(\chi_{x}, \chi_{y})} \{ \mathbb{E}_{\chi_{x}, \chi_{y}} c(\chi_{x}, \chi_{y}) : \chi_{x} \sim \alpha, \chi_{y} \sim \beta \}
\end{align}

If $\chi$ is associated with some metric $d_{\chi}$, we can formulat eour ground cost as $c(x,y) = d_{\chi}(x,y)^{p}$ for $p \geq 1$. Under this condition, we can define the p-Wasserstein as $\text{W}_{p}(\alpha, \beta) = \text{OT}(\alpha, \beta)^{1/p}$


## Sinkhorn Divergence

For computational optimal transport, we are dealing with finite samples $x_{1}, ..., x_{n}$, $y_{1}, ..., y_{m}$. Our cost matrix is then a $n \times m$ matrix. 

In thise case, solving the optimal transport problem becomes a linear optimization program that scales cubically with sample size, often becoming computationally prohibative. We can make the problem more computationally tractable by adding entropy regularization. This allows us to calculate approximate transport plans much more easily.

The problem becomes:

\begin{align}
\text{OT}_{\epsilon}(\alpha, \beta) ~=~ \min_{x\in\Pi(\alpha, \beta)}\int_{\chi_{x} \times \chi_{y}} c(x,y)d\pi(x,y) + \epsilon \text{H}(\pi | \alpha \otimes \beta)
\end{align}

\begin{align}
\text{H}(\pi | \alpha \otimes \beta) ~=~ \int log(\frac{d\pi}{d\alpha d\beta})d\pi
\end{align}

Where $\epsilon$ is a constant that determines the strength of the entropic regularization.

Since we are dealing with discrete samples, we can also phrase these equations in terms of summations.

\begin{align}
\text{OT}_{\epsilon}(\alpha, \beta) ~=~ \min_{\pi_{i,j}} \sum_{i,j} \pi_{i,j} C_{i,j} + \epsilon \text{KL}(\pi, \alpha \otimes \beta)
\end{align}

\begin{align}
\text{KL}(\pi, \alpha \otimes \beta) ~=~ \sum_{i,j} [\pi_{i,j} log \frac{\pi_{i,j}}{\alpha_{i} \beta_{j}} - \pi_{i,j} + \alpha_{i} \beta_{j}]
\end{align}

With entropic regularization, we can define the __Sinkhorn Divergence__.

\begin{align}
\text{SD}_\epsilon(\alpha,\beta)~=~ \text{OT}_\varepsilon(\alpha,\beta)
~-~\tfrac{1}{2}\text{OT}_\varepsilon(\alpha,\alpha)
~-~\tfrac{1}{2}\text{OT}_\varepsilon(\beta,\beta).
\end{align}

The Sinkhorn Divergence allows us to calculate a fuzzy transport plan that is much more computationally tractable than the unregularized optimal transport problem. The regularization strength $\epsilon$ allows us to control how fuzzy the transport plan is. Specifically, $\epsilon$ allows Sinkhorn Divergences to interpolate between Optimal Transport and kernel norms.

\begin{align}
\text{OT}_C(\alpha,\beta)
~~ \xleftarrow{0\leftarrow \epsilon} ~~
\text{S}_\epsilon(\alpha,\beta)
~~ \xrightarrow{\epsilon\rightarrow +\infty} ~~
\tfrac{1}{2}\|\alpha-\beta\|_{-C}^2.
\end{align}

Small values of $\epsilon$ give a more computationally expensive problem that is closer to Optimal Transport. Large $\epsilon$ values give a computationally cheap problem that is closer to kernel norms.

## Sinkhorn Algorithm

How do we actually compute the transport plan? We can do this with the Sinkhorn algorithm. We start by decomposing 

\begin{align}
\text{C}_{i,j} + \epsilon ~ log \frac{\pi_{i,j}}{\alpha_{i} \beta_{j}} ~=~ f_{i} + g_{j}
\end{align}

Which would give us the transport plan as 

\begin{align}
\pi ~=~ \exp\frac{1}{\epsilon}(f\oplus g-C)\,\cdot\,\alpha\otimes\beta
\end{align}

\begin{align}
\pi ~=~ \text{diag}(\alpha_iU_i)\,K_{i,j}\,\text{diag}(V_j\beta_j)
\end{align}

\begin{align}
U_i ~=~ \text{exp}(f_i / \epsilon), \quad V_j ~=~ \text{exp}(g_j / \epsilon), \quad K_{i,j} ~=~ \text{exp}(-\text{C}_{i,j} / \epsilon)
\end{align}

With this setup, we can compute the Sinkhorn algorithm. The Sinkhorn algorithm iteratively updates

\begin{align}
f_i \leftarrow \alpha_i \oslash K_{i,j} g_j
\end{align}

\begin{align}
g_j \leftarrow \beta_j \oslash K_{i,j} f_i
\end{align}


For numerical stability, we can express the vectors $f_i$, $g_j$ in log form as 

\begin{align}
f_i = \epsilon ~ logU, \quad g_j = \epsilon ~ logV
\end{align}

\begin{align}
U ~=~ \frac{1}{(K(V\beta))}, ~~~~ V ~=~ \frac{1}{(K^T(U\alpha))}.
\end{align}

We can then define our Sinkhorn iterations as

\begin{align}
g_j^{(n+1)}~&=~
-\varepsilon \log K^T(U^{(n)}\alpha)\\
~&=~
-\varepsilon \log \sum_{i=1}^\text{N} \exp \big( - \varepsilon ~ c(x,y)+f_i^{(n)}/\varepsilon +\log\alpha_i \big)\\~\\
f_i^{(n+1)}~&=~
-\varepsilon \log K(V^{(n+1)}\beta)\\
~&=~
-\varepsilon \log \sum_{j=1}^\text{M} \exp \big( -\varepsilon~c(x,y)+g_j^{(n+1)}/\varepsilon +\log\beta_i \big)
\end{align}

## Gaussian Approximation

For large datasets, even the Sinkhorn algorithm is computationally infeasible. In this case, we can approximate the transport cost by assuming our measures $\chi_{x}$, $\chi_{y}$ are multivariate Gaussian distributions given by $\chi_{x} \sim N(\mu_{x}, \Sigma_{x})$. 

We can then calculate the 2-Wasserstein distance between our measures with the closed form solution:

\begin{align}
\text{W}^2_{2}(\alpha, \beta) = \| \mu_{\alpha} - \mu_{\beta} \|^{2}_{2} + \| \Sigma^{1/2}_{\alpha} - \Sigma^{1/2}_{\beta} \|^{2}_{F}
\end{align}

Sources:

[Gradient Flows Between Sampled Measires](https://www.math.ens.fr/~feydy/Teaching/DataScience/gradient_flows.html)

[Computational Optimal Transport](https://arxiv.org/pdf/1803.00567.pdf)

[Geometric Dataset Distances via Optimal Transport](https://arxiv.org/pdf/2002.02923.pdf)
